# Spot/short grid search example

See the earlier notebook for more information.

### Grid search target strategy loading


In [1]:
import datetime
import importlib.util
import sys

from tradeexecutor.utils.default_strategies import get_default_strategies_path

strategies_folder = get_default_strategies_path()
strategy_path = strategies_folder / "spot-and-short-momentum-grid-3.py"

# See https://stackoverflow.com/a/67692/315168
spec = importlib.util.spec_from_file_location("strategy_mod", strategy_path)
python_mod = importlib.util.module_from_spec(spec)
sys.modules["strategy_mod"] = python_mod
spec.loader.exec_module(python_mod)

print(f"Backtesting for strategy {strategy_path}, engine version {python_mod.trading_strategy_engine_version}")

Backtesting for strategy /Users/moo/code/ts/trade-executor/strategies/spot-and-short-momentum-grid-3.py, engine version 0.3


### Set up the market data client



In [2]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


Load the strategy universe and analyse what trading pairs the strategy code gives for us.

In [3]:
from tradeexecutor.strategy.execution_context import notebook_execution_context

universe = python_mod.create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    python_mod.universe_options,
)

## Available trading universe

Inspect trading pairs and lending markets we are going to backtest.


In [4]:
from IPython.display import HTML

from tradeexecutor.analysis.universe import analyse_long_short_universe
from tradingstrategy.utils.jupyter import format_links_for_html_output

df = analyse_long_short_universe(universe)
df = format_links_for_html_output(df, ("Price data page", "Lending rate page",))

display(HTML(df.to_html(escape=False)))

,Lending asset,Stablecoin,Best trading pair,Lending available at,Trading available at,Price data page,Lending rate page
0,BAL,no,No AMM pools found,2022-03-16,-,View,View
1,LINK,no,LINK-USDC at 5 BPS fee tier on Uniswap v3,2022-03-16,2022-08-21,View,View
2,USDC,yes,No AMM pools found,2022-03-16,-,View,View
3,WETH,no,WETH-USDC at 5 BPS fee tier on Uniswap v3,2022-03-16,2022-07-07,View,View
4,WMATIC,no,WMATIC-USDC at 5 BPS fee tier on Uniswap v3,2022-03-12,2022-07-07,View,View


## Run grid search

In [5]:
from pathlib import Path
from tradeexecutor.backtest.grid_search import perform_grid_search, prepare_grid_combinations

# This is the path where we keep the result files around
storage_folder = Path(f"/tmp/{strategy_path.stem}-grid-search")

parameters = {
    "cycle_duration_days": [7],
    "momentum_lookback": [7],
    "take_profit": [1.07],
    "negative_take_profit": [1.07],
    "positive_mometum_threshold": [0.01],
    "negative_mometum_threshold": [-0.05],
    "mod_adjust": [0, 1, 2, 3, 4, 5, 6, 7, 8],
}
combinations = prepare_grid_combinations(parameters, storage_folder, clear_cached_results=True)

grid_search_results = perform_grid_search(
    python_mod.grid_search_worker,
    universe,
    combinations,
    max_workers=4,
)


  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

  0%|          | 0/57801600 [00:00<?, ?it/s]

## Examine grid search results


In [6]:
from tradeexecutor.analysis.grid_search import analyse_grid_search_result
from tradeexecutor.analysis.grid_search import visualise_table
table = analyse_grid_search_result(grid_search_results)

visualise_table(table)